# Visualising near-wall coherent structures

This tutorial showcases Pyvista's ability to directly access the underlying data giving user's significant flexibility. The data has been gzipped to bypass file size limitations on GitHub. To decompress

```bash
gzip -d turbChannel/*.gz
```

## Turbulent channel flow
In this example we use the doubly periodic turbulent channel flow example from nekRS. This has been run at friction Reynolds number, $Re_\tau=550$ (don't worry). We will be visulising some of the important near wall coherent turbulent structures to showcase Pyvista's ability.

In [1]:
import pyvista as pv
# Load the data
fluid_reader = pv.get_reader('turbChannel/turbChannel.nek5000')
fluid_reader.enable_merge_points()
fluid_reader.set_active_time_point(fluid_reader.number_time_points-1)

fluid = fluid_reader.read()
fluid.plot(window_size=(450,450),
           scalars='Velocity',
           cmap='bwr')

3D-Mesh found, spectral element of size = 8*8*8=512


Widget(value='<iframe src="http://localhost:40369/index.html?ui=P_0x76e799e43550_0&reconnect=auto" class="pyvi…

## Point data and cell data
From Paraview, you should already be familiar with point data and cell data
- Point data: arrays defined at the cell vertices.
- Cell data: arrays defined in the element.

In Pyvista, they can be accessed using `point_data` and `cell_data` with different arrays accessed by key.

In [2]:
print(fluid.point_data)

for key in fluid.point_data.keys():
    print(key,
          type(fluid.point_data[key]).__name__,
          fluid.point_data[key].shape)

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : Velocity-normed
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    Velocity                float32    (3630803, 3)
    Velocity Magnitude      float32    (3630803,)
    Pressure                float32    (3630803,)
    Velocity-normed         float32    (3630803,)           SCALARS
Velocity pyvista_ndarray (3630803, 3)
Velocity Magnitude pyvista_ndarray (3630803,)
Pressure pyvista_ndarray (3630803,)
Velocity-normed pyvista_ndarray (3630803,)


## $\lambda_2$ vortex indentification criteria

Vortices represent one of the most important and widely recognised features of turbulent flows. In this tutorial, we show these using the $\lambda_2$ vortex identification criterion as it provides an ideal opportunity to show manipulation of the underlying data in Pyvista and its compatability with numpy. Jeong & Hussein (1995) defined a vortex to be where the second largest eigenvalue, $\lambda_2<0$ of the matrix

$S_{ik}S_{kj} + \Omega_{ik}\Omega_{kj},$

where $S_{ij}$ is the strain rate tensor and $\Omega_{ij}$ is the rotational rate tensor defined as 

$S_{ij}=\frac{1}{2}\left(\frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i}\right)$

$\Omega_{ij}=\frac{1}{2}\left(\frac{\partial u_i}{\partial x_j} - \frac{\partial u_j}{\partial x_i}\right)$

First we need to compute the gradient...

In [3]:
fluid = fluid.compute_derivative(scalars='Velocity',
                                 qcriterion='qcrit')

print(fluid.point_data)

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : Velocity-normed
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    Velocity                float32    (3630803, 3)
    Velocity Magnitude      float32    (3630803,)
    Pressure                float32    (3630803,)
    Velocity-normed         float32    (3630803,)           SCALARS
    gradient                float32    (3630803, 9)
    qcrit                   float32    (3630803,)


We need to compute $S_{ij}$ and $\Omega_{ij}$. Note that the velocity gradient tensor is arranges as (XX, XY, XZ, YX, YY, YZ, ZX, ZY, ZZ)

In [4]:
import numpy as np
dudx = fluid.point_data['gradient'].reshape((fluid.n_points,3,3))
dudxT = np.transpose(dudx, axes=(0,2,1))

S = 0.5*(dudx + dudxT)
Omega = 0.5*(dudx - dudxT)

A = np.matmul(S, S) + np.matmul(Omega, Omega)

Calculate the second largest eigenvalue using `np.linalg.eigh`

In [5]:
lambda_, _ = np.linalg.eigh(A)
fluid.point_data['lambda2'] = lambda_[:,1]

### Plotting the isosurface

First, we need to create the isosurface which can be done using the `contour` filter

In [6]:
lambda2_contour = fluid.contour(scalars='lambda2',
                                isosurfaces=[-2])

Q_contour = fluid.contour(scalars='qcrit',
                          isosurfaces=[2])

We will now plot both using linked views

In [7]:
# Create two side-by-side render windows
p = pv.Plotter(window_size=(800,400), shape=(1,2), border=False)

#select first render window and plot the element blocks
p.subplot(0,0)
p.add_mesh(lambda2_contour,
           scalars='Velocity',
           cmap='bwr')

#select second render window and plot each side set with a different color

p.subplot(0,1)
p.add_mesh(Q_contour,
           scalars='Velocity',
           cmap='bwr')

#link views and adjust camera
p.link_views()
p.view_zy()
p.camera.azimuth = 30
p.camera.elevation = 20
p.show()

Widget(value='<iframe src="http://localhost:40369/index.html?ui=P_0x76e709c7d5a0_1&reconnect=auto" class="pyvi…

## Visualising near wall streaks

Another important near-wall coherent structure are the alternating low and high speed streaks that are found near the wall. This can visualised using isosurfaces of the streamwise fluctuating velocity, $u'=u -\bar{u}$. While NekRS gives us the velocity, we must calculate its mean. 

Given the statistics of turbulent channel flow are streamwise/spanwise homogeneous, we can compute the average in the average in these directions. While the data from NekRS is unstructured, the grid itself is structured, which we can recover.

First, we must find the unique coordinates of the points in the $y$ direction, noting that the polynomial order of the case is 7 and there are 16 elements in the $y$ direction giving an expected number of points as $7\times 16 +1$

In [8]:
y_points = fluid.points[:, 1]
y = np.unique(y_points)

assert y.size == 7*16+1

Now, we will loop through the values of `y` and average $u$ where `y_points == y`. We then subtract this from $u$ to get the $u'$. This is then assigned to the fluid data.

In [9]:
uprime = np.zeros((fluid.n_points))
for i in range(y.size):
    mask = y_points == y[i]

    u = fluid.point_data['Velocity'][mask, 0]
    u_mean = u.mean()
    uprime[mask] = u - u_mean

fluid.point_data["u'"] = uprime

Similar to the vortices, these will be visualised using the `contour` method. In this case, for clarity we will show only the streaks on the bottom wall, which is located at $y=-1$. This is achieved by using the `clip` method.

In [10]:
fluid_clipped = fluid.clip(normal='y',
                           origin=(0,-0.85,0))

low_speed = fluid_clipped.contour(scalars="u'",
                          isosurfaces=[-0.15])

high_speed = fluid_clipped.contour(scalars="u'",
                          isosurfaces=[0.15])

Now we can plot both with different colours for low-speed and high-speed streaks. To show the wall, we will also add a floor.

In [11]:
# Create two side-by-side render windows
p = pv.Plotter(window_size=(800,400))

#select first render window and plot the element blocks
p.add_mesh(low_speed, color='g')
p.add_mesh(high_speed, color='b')
p.add_floor(face='-y')

p.view_zy()
p.camera.azimuth = 30
p.camera.elevation =20
p.add_axes()
p.show()

Widget(value='<iframe src="http://localhost:40369/index.html?ui=P_0x76e709c7c2e0_2&reconnect=auto" class="pyvi…